In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

from tqdm import tqdm

from plyfile import PlyData, PlyElement
import mesh_to_depth as m2d

In [56]:
max_depth = 0
min_depth = 1000
for mesh_id in tqdm(range(1000)):
    # read mesh
    plydata = PlyData.read(f'../data/1kmeshes/mesh{mesh_id}.ply')

    # Load triangle mesh data. See python/resources/airplane/models/model_normalized.obj
    vertices = np.zeros((len(plydata['vertex']), 3), dtype=np.float32)  # An array of shape (num_vertices, 3) and type np.float32.
    for i in range(len(plydata['vertex'])):
        vertices[i] = np.array([plydata['vertex'][i][0], plydata['vertex'][i][1], plydata['vertex'][i][2]])

    faces = np.zeros((len(plydata['face']['vertex_indices']), 3), dtype=np.uint32)  # An array of shape (num_faces, 3) and type np.uint32.
    for i in range(len(plydata['face'])):
        faces[i] = plydata['face']['vertex_indices'][i]

    # camera settings
    params = []
    params.append({
        'cam_pos': [0, 20, 500], 'cam_lookat': [0, 20, 0], 'cam_up': [0, 1, 0],
        'x_fov': 0.5236,  # End-to-end field of view in radians 45 / 180
        'near': 0.1, 'far': 2000,
        'height': 512, 'width': 512,
        'is_depth': True,  # If false, output a ray displacement map, i.e. from the mesh surface to the camera center.
    })

    # render & show depth map
    depth_maps = m2d.mesh2depth(vertices, faces, params, empty_pixel_value=np.nan)
    depth_maps[0] = depth_maps[0] - 325
    _min_depth = np.nan_to_num(depth_maps[0], nan=1000).min()
    depth_maps[0] = np.nan_to_num(depth_maps[0])
    depth_maps[0] = cv2.resize(depth_maps[0],(64,64))
    assert((depth_maps[0]>=0).sum() == 64*64)
    # record max/min depth
    _max_depth = depth_maps[0].max()
    if _max_depth>max_depth:
        max_depth = _max_depth
    if _min_depth<min_depth:
        min_depth = _min_depth
    # # visualize
    # plt.imshow(depth_maps[0], interpolation='none')
    # plt.colorbar()
    # plt.show()

    # save depth map
    cv2.imwrite(f"../data/1kmeshes_depth_map/mesh{mesh_id}_dm.png", depth_maps[0])

100%|██████████| 1000/1000 [1:43:52<00:00,  6.23s/it]


In [58]:
min_depth, max_depth

(28.11789, 212.50552)

# create textured glb

## read color

In [1]:
import scipy.io
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
import os

In [2]:
# read mat files
AppleMesh00_pt = scipy.io.loadmat('../../data/face_data/create_textured_glb/AppleMesh00_pt.mat')
AppleMesh01_pt = scipy.io.loadmat('../../data/face_data/create_textured_glb/AppleMesh01_pt.mat')
face_parts = scipy.io.loadmat('../../data/face_data/create_textured_glb/FacePartsInd.mat')

In [53]:
# check FacePartsInd
# a list of 273 index of vertices, presumably the subset of the 1220 vertices from ARKit 
face_parts['FacePartsInd'][0][0][0].shape, face_parts['FacePartsInd'][0][0][0].max()

((273, 1), 1207)

In [49]:
# check AppleMesh00_pt
# they are triangles, shape (6284, 3), values denote the index of vertices
# num of vertices is 3182
AppleMesh00_pt['AppleMesh00_pt'][0][0][0].shape, AppleMesh00_pt['AppleMesh00_pt'][0][0][0]

((6284, 3),
 array([[   1,    2,    3],
        [   1,    3,    6],
        [   1,    6,    7],
        ...,
        [ 534, 2806,  526],
        [2805,  543,  552],
        [2805, 2806,  543]], dtype=uint16))

In [52]:
AppleMesh00_pt['AppleMesh00_pt'][0][0][0].max()

3182

In [5]:
with open('../../data/face_data/create_textured_glb/faceData1.txt') as f:
    AppleMesh00_face_data = f.readlines()

In [70]:
len(AppleMesh00_face_data)

106

In [68]:
# check faceData.txt from ARKit
# 106 lines -> 106 images
# each line: [2:-52] -> 1220 vertices 
vertices = np.zeros((1220, 3))
img_id = 1
assert(len(AppleMesh00_face_data[img_id].split("~")) == 1274)
for v_index, vertex in enumerate(AppleMesh00_face_data[img_id].split("~")[2:-52]):
    _splits = vertex.split(":")
    assert(len(_splits)==3)
    vertices[v_index][0] = _splits[0]
    vertices[v_index][1] = _splits[1]
    vertices[v_index][2] = _splits[2]

In [72]:
vertices[:10]

array([[ 1.2661343e-04, -2.9175520e-02,  6.2786760e-02],
       [ 9.7752220e-06, -3.2182045e-02,  6.3591830e-02],
       [ 2.6316586e-04, -2.5501460e-02,  6.2852114e-02],
       [ 3.7184960e-04, -2.2400688e-02,  6.3982480e-02],
       [ 4.6176204e-04, -1.9778693e-02,  6.5859820e-02],
       [ 5.5558980e-04, -1.7112002e-02,  7.1387930e-02],
       [ 6.3899544e-04, -1.3283982e-02,  7.7885990e-02],
       [ 6.9648586e-04, -8.6886480e-03,  8.0109200e-02],
       [ 7.5923190e-04, -2.3595237e-03,  7.9672270e-02],
       [ 7.9956430e-04,  3.2384754e-03,  7.6325860e-02]])

In [ ]:
### how to map these back to the image?

In [69]:
vertices[:,0].max(), vertices[:,0].min(), vertices[:,1].max(), vertices[:,1].min(), vertices[:,2].max(), vertices[:,2].min()

(0.07629348, -0.07637527, 0.090478584, -0.09703311, 0.0801092, -0.028006377)

## write color

In [73]:
from pygltflib import GLTF2, BufferFormat
from pygltflib.utils import ImageFormat

In [91]:
# check glb file
gltf = GLTF2().load("../../data/face_data/create_textured_glb/mesh_Baselmesh0.glb")
# gltf = GLTF2().load("../../data/face_data/create_textured_glb/mesh_AppleMesh00applefacepotatoheadnormalhires.glb")

In [92]:
gltf

GLTF2(extensions={}, extras={}, accessors=[Accessor(extensions={}, extras={}, bufferView=0, byteOffset=0, componentType=5126, normalized=False, count=65535, type='VEC3', sparse=None, max=[86.18408966064453, 121.35191345214844, 91.73141479492188], min=[-2.168947696685791, -71.36751556396484, -128.45411682128906], name=None), Accessor(extensions={}, extras={}, bufferView=1, byteOffset=0, componentType=5126, normalized=False, count=65535, type='VEC3', sparse=None, max=[], min=[], name=None), Accessor(extensions={}, extras={}, bufferView=2, byteOffset=0, componentType=5126, normalized=False, count=65535, type='VEC4', sparse=None, max=[], min=[], name=None), Accessor(extensions={}, extras={}, bufferView=3, byteOffset=0, componentType=5123, normalized=False, count=65535, type='SCALAR', sparse=None, max=[], min=[], name=None), Accessor(extensions={}, extras={}, bufferView=4, byteOffset=0, componentType=5126, normalized=False, count=65535, type='VEC3', sparse=None, max=[86.1733627319336, 121.1

In [93]:
mesh = gltf.meshes[gltf.scenes[gltf.scene].nodes[0]]

In [95]:
mesh.primitives

[Primitive(extensions={}, extras={}, attributes=Attributes(POSITION=0, NORMAL=1, TANGENT=None, TEXCOORD_0=None, TEXCOORD_1=None, COLOR_0=2, JOINTS_0=None, WEIGHTS_0=None), indices=3, mode=4, material=None, targets=[]),
 Primitive(extensions={}, extras={}, attributes=Attributes(POSITION=4, NORMAL=5, TANGENT=None, TEXCOORD_0=None, TEXCOORD_1=None, COLOR_0=6, JOINTS_0=None, WEIGHTS_0=None), indices=3, mode=4, material=None, targets=[]),
 Primitive(extensions={}, extras={}, attributes=Attributes(POSITION=7, NORMAL=8, TANGENT=None, TEXCOORD_0=None, TEXCOORD_1=None, COLOR_0=9, JOINTS_0=None, WEIGHTS_0=None), indices=3, mode=4, material=None, targets=[]),
 Primitive(extensions={}, extras={}, attributes=Attributes(POSITION=10, NORMAL=11, TANGENT=None, TEXCOORD_0=None, TEXCOORD_1=None, COLOR_0=12, JOINTS_0=None, WEIGHTS_0=None), indices=3, mode=4, material=None, targets=[]),
 Primitive(extensions={}, extras={}, attributes=Attributes(POSITION=13, NORMAL=14, TANGENT=None, TEXCOORD_0=None, TEXCOOR

In [94]:
import struct

# get the vertices for each primitive in the mesh (in this example there is only one)
for primitive in mesh.primitives:

    # get the binary data for this mesh primitive from the buffer
    accessor = gltf.accessors[primitive.attributes.POSITION]
    bufferView = gltf.bufferViews[accessor.bufferView]
    buffer = gltf.buffers[bufferView.buffer]
    data = gltf.get_data_from_buffer_uri(buffer.uri)

    # pull each vertex from the binary buffer and convert it into a tuple of python floats
    vertices = []
    for i in range(accessor.count):
        index = bufferView.byteOffset + accessor.byteOffset + i*12  # the location in the buffer of this vertex
        d = data[index:index+12]  # the vertex data
        v = struct.unpack("<fff", d)   # convert from base64 to three floats
        vertices.append(v)
        # print(i, v)

    # convert a numpy array for some manipulation
    S = np.array(vertices)
    print(S.shape)

(65535, 3)
(65535, 3)
(65535, 3)
(65535, 3)
(65535, 3)
(20805, 3)


In [96]:
65535/20805

3.1499639509733237

In [90]:
S.shape

(3182, 3)

In [ ]:
# save new mesh
gltf.save("./new_AppleMesh00.gltf")